In [11]:
import pandas as pd
import numpy as np
import folium
import requests

from bs4 import BeautifulSoup


from tqdm.notebook import tqdm 
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service


from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [12]:

chrome_options = webdriver.ChromeOptions()

chrome_path = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe" 

chrome_options.binary_location = chrome_path  

drive_path = 'chromedriver.exe' 
myservice = Service(drive_path) 
driver = webdriver.Chrome(service=myservice, options=chrome_options) 
print(type(driver)) # 객체가 잘 생성되었는 지 확인

wait_time = 10 
driver.implicitly_wait(wait_time)

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [13]:
driver.maximize_window() 

In [14]:

days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
base_url = "https://comic.naver.com/webtoon?tab="
driver.get(base_url) 

In [15]:

week_days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
all_webtoons = []

for idx, day in enumerate(week_days, start=2):
  
    naver_webtoon_weekday = f'#wrap > header > div.SubNavigationBar__snb_wrap--A5gfM > nav > ul > li:nth-child({idx}) > a'
    a_tag = driver.find_element(By.CSS_SELECTOR, naver_webtoon_weekday)

    print('a_tag.text:',a_tag.text) 
    
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, naver_webtoon_weekday))).click()
   

   
    ul_element = driver.find_element(By.CSS_SELECTOR, "#content > div:nth-child(1) > ul") 
    
    li_elements = ul_element.find_elements(By.TAG_NAME, "li")

    with open(f"{day}_li_elements.html", "w", encoding="utf-8") as f:
        for idx, li in enumerate(li_elements, start=1):
          
            f.write(f"[{idx}]\n")
            f.write(li.get_attribute("outerHTML"))          
            f.write("\n\n")
    print("저장 완료")

    
    import os
    import csv

    records = []


    for idx, li in enumerate(tqdm(li_elements, desc=f"{day} 웹툰 크롤링 진행"), start=1):
        
        if idx == 10 :
            break

        os.makedirs(day, exist_ok=True)
        
        csv_filename = os.path.join(day, f"{day}.csv") 

        file_exists = os.path.exists(csv_filename)
    
        csv_file = open(csv_filename, mode='a', newline='', encoding='utf-8-sig')
        writer = csv.writer(csv_file) 

        
        
        if not file_exists:
            writer.writerow(["제목", "작가", "평점", "이미지파일명", "링크"])
            
        try:
            title = driver.find_element(By.CSS_SELECTOR, f"#content > div:nth-child(1) > ul > li:nth-child({idx}) > div > a > span > span").text.strip()
              
            author = li.find_element(By.CSS_SELECTOR, ".ContentAuthor__author--CTAAP").text.strip()
            rating = li.find_element(By.CSS_SELECTOR, ".Rating__star_area--dFzsb .text").text.strip()
            
            img = li.find_element(By.CSS_SELECTOR, "img.Poster__image--d9XTI").get_attribute("src")
            
            link = li.find_element(By.CSS_SELECTOR, "a.Poster__link--sopnC").get_attribute("href")
           
            img_url = li.find_element(By.CSS_SELECTOR, "img.Poster__image--d9XTI").get_attribute("src")
            
            print(f"[{idx}] {title}")
            print(f" - 작가: {author}")
            print(f" - 평점: {rating}")
            print(f" - 이미지: {img}")
            print(f" - 링크: {link}")
          
            img_tag = li.find_element(By.CSS_SELECTOR, "img.Poster__image--d9XTI")
            img_url = img_tag.get_attribute("src") # 여기는 data-src속성이 없으니까 src로만 해도 될것 같다. 
            

            img_filename = os.path.join(day, f"{idx}_{title.replace('/', '_')}.jpg")
            
            print('img_filename:',img_filename)

            
            try:
                headers = {'User-Agent': 'Mozilla/5.0'}
                img_data = requests.get(img, headers=headers, timeout=10).content

                with open(img_filename, "wb") as f:
                    f.write(img_data) 
                    
            except Exception as e:
                print(f"이미지 다운로드 실패: {e}")
                img_filename = ""
            
            
            records.append({
                "제목": title,
                "작가": author,
                "평점": rating,
                "이미지파일": os.path.basename(img_filename), 
                "링크": link
            })

            writer.writerow([title, author, rating, os.path.basename(img_filename), link]) # mon.csv, tue.csv 
            
        except Exception as e:
            print(f"[{idx}] {title}항목 추출 중 오류 발생: {e}") 
            print(li_elements)
            continue
            
    print('records:',records)
    
    df_day = pd.DataFrame(records)
    csv_path = os.path.join(day, f"{day}.csv")
    df_day.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"{csv_path} 저장 완료 ({len(df_day)}개 항목)")

    all_webtoons.extend(records)
    
df_all = pd.DataFrame(all_webtoons)
df_all.to_csv("all_webtoons.csv", index=False, encoding="utf-8-sig")
print("전체 웹툰 CSV 파일 저장 완료: all_webtoons.csv")


a_tag.text: 월
저장 완료


mon 웹툰 크롤링 진행:   0%|          | 0/107 [00:00<?, ?it/s]

[1] 만남어플 중독
 - 작가: 루즌아
 - 평점: 9.84
 - 이미지: https://image-comic.pstatic.net/webtoon/839004/thumbnail/thumbnail_IMAG21_258e9101-6b52-49a2-bcaa-6f90f540624f.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=839004&tab=mon
img_filename: mon\1_만남어플 중독.jpg
[2] 참교육
 - 작가: 채용택 / 한가람
 - 평점: 9.89
 - 이미지: https://image-comic.pstatic.net/webtoon/758037/thumbnail/thumbnail_IMAG21_bbe528c8-7b30-4ea9-b06f-22122ce7d180.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=758037&tab=mon
img_filename: mon\2_참교육.jpg
[3] 환생천마
 - 작가: JP / 부겸 / 장영훈
 - 평점: 9.95
 - 이미지: https://image-comic.pstatic.net/webtoon/822657/thumbnail/thumbnail_IMAG21_99e49512-e05d-48c3-846d-d898f78523df.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=822657&tab=mon
img_filename: mon\3_환생천마.jpg
[4] 왕과의 야행
 - 작가: 보리
 - 평점: 9.98
 - 이미지: https://image-comic.pstatic.net/webtoon/817859/thumbnail/thumbnail_IMAG21_7479316a-1fca-40d6-9d8e-455f8b2fb0db.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=817859&tab=mon
img_

tue 웹툰 크롤링 진행:   0%|          | 0/50 [00:00<?, ?it/s]

[1] 괴력 난신
 - 작가: 매드버드 / 김태형 / 한중월야
 - 평점: 9.95
 - 이미지: https://image-comic.pstatic.net/webtoon/821597/thumbnail/thumbnail_IMAG21_78e9abac-677f-4127-9914-93cf45cab47a.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=821597&tab=tue
img_filename: tue\1_괴력 난신.jpg
[2] 오! 나의 교주님
 - 작가: 김꿀빨, 김완두
 - 평점: 8.85
 - 이미지: https://image-comic.pstatic.net/webtoon/841624/thumbnail/thumbnail_IMAG21_154d7538-16dc-4aef-acb4-60d31e2834c8.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=841624&tab=tue
img_filename: tue\2_오! 나의 교주님.jpg
[3] 절대회귀
 - 작가: JP / 박진환 / 장영훈
 - 평점: 9.96
 - 이미지: https://image-comic.pstatic.net/webtoon/828715/thumbnail/thumbnail_IMAG21_8957548a-2eb2-4ddf-a50e-dbc7fddc16a7.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=828715&tab=tue
img_filename: tue\3_절대회귀.jpg
[4] 김부장
 - 작가: 박만사, 남자의 이야기 / 정종택
 - 평점: 9.64
 - 이미지: https://image-comic.pstatic.net/webtoon/783053/thumbnail/thumbnail_IMAG21_1135578a-87ec-48ba-b2c5-c7b93b21f91c.jpg
 - 링크: https://comic.naver.com/webto

...

[1] 오! 나의 교주님
 - 작가: 김꿀빨, 김완두
 - 평점: 8.85
 - 이미지: https://image-comic.pstatic.net/webtoon/841624/thumbnail/thumbnail_IMAG21_154d7538-16dc-4aef-acb4-60d31e2834c8.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=841624&tab=fri
img_filename: fri\1_오! 나의 교주님.jpg
[2] 광마회귀
 - 작가: JP / 이히 / 유진성
 - 평점: 9.97
 - 이미지: https://image-comic.pstatic.net/webtoon/776601/thumbnail/thumbnail_IMAG21_7365135131254864183.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=776601&tab=fri
img_filename: fri\2_광마회귀.jpg
[3] 역대급 영지 설계사
 - 작가: 이현민 / 김현수 / 문백경
 - 평점: 9.98
 - 이미지: https://image-comic.pstatic.net/webtoon/777767/thumbnail/thumbnail_IMAG21_cc85f891-272b-450a-b642-cffe1568ab71.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=777767&tab=fri
img_filename: fri\3_역대급 영지 설계사.jpg
[4] 나 혼자 만렙 뉴비
 - 작가: WAN.Z / 스윙뱃 / 메슬로우
 - 평점: 9.94
 - 이미지: https://image-comic.pstatic.net/webtoon/773797/thumbnail/thumbnail_IMAG21_37c6a85a-d3a6-4afc-909a-420973f63019.jpg
 - 링크: https://comic.naver.com/webtoon/

sat 웹툰 크롤링 진행:   0%|          | 0/50 [00:00<?, ?it/s]

[1] 개꿈
 - 작가: 신송림
 - 평점: 9.76
 - 이미지: https://image-comic.pstatic.net/webtoon/818791/thumbnail/thumbnail_IMAG21_a613db8b-8319-4bfe-8971-1df89f4df979.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=818791&tab=sat
img_filename: sat\1_개꿈.jpg
[2] 호랑이형님
 - 작가: 이상규
 - 평점: 9.88
 - 이미지: https://image-comic.pstatic.net/webtoon/650305/thumbnail/thumbnail_IMAG21_9e070729-5990-4653-90dd-1158847c1c68.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=650305&tab=sat
img_filename: sat\2_호랑이형님.jpg
[3] 시든 꽃에 눈물을
 - 작가: 개
 - 평점: 9.96
 - 이미지: https://image-comic.pstatic.net/webtoon/827190/thumbnail/thumbnail_IMAG21_ce05e060-9420-4494-a004-278f90e8dde9.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=827190&tab=sat
img_filename: sat\3_시든 꽃에 눈물을.jpg
[4] 마도전생기
 - 작가: 포스스튜디오 / codezero
 - 평점: 9.92
 - 이미지: https://image-comic.pstatic.net/webtoon/807777/thumbnail/thumbnail_IMAG21_e47489ea-bf97-4726-a8e7-6403092a303a.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=807777&tab=sat
img_

sun 웹툰 크롤링 진행:   0%|          | 0/50 [00:00<?, ?it/s]

[1] 육아일기
 - 작가: 자까
 - 평점: 9.96
 - 이미지: https://image-comic.pstatic.net/webtoon/812354/thumbnail/thumbnail_IMAG21_c20b50b3-8cbd-4f14-a8f8-5a67e5cacea0.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=812354&tab=sun
img_filename: sun\1_육아일기.jpg
[2] 입학용병
 - 작가: YC / 락현
 - 평점: 9.90
 - 이미지: https://image-comic.pstatic.net/webtoon/758150/thumbnail/thumbnail_IMAG21_4135492154714961716.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=758150&tab=sun
img_filename: sun\2_입학용병.jpg
[3] 무직백수 계백순
 - 작가: 지발
 - 평점: 9.86
 - 이미지: https://image-comic.pstatic.net/webtoon/811721/thumbnail/thumbnail_IMAG21_5e1f571d-1c76-4b2a-b8b1-742865ab82f9.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=811721&tab=sun
img_filename: sun\3_무직백수 계백순.jpg
[4] 2회차 환관이 남성을 되찾음
 - 작가: LICO / 포스스튜디오 / 노빠꾸맨
 - 평점: 9.87
 - 이미지: https://image-comic.pstatic.net/webtoon/832566/thumbnail/thumbnail_IMAG21_dc3db5f7-a4ab-45fe-b261-92f16d3478bc.jpg
 - 링크: https://comic.naver.com/webtoon/list?titleId=832566&tab=sun
img_